In [41]:
# Cell 1: Markdown
"""
# COVID-19 Data Preparation and EDA
*By: Mohit (Data Lead)*

## Objective: 
- Load and clean COVID-19 data
- Transform data structure
- Perform exploratory data analysis
- Calculate key statistics
"""

'\n# COVID-19 Data Preparation and EDA\n*By: Mohit (Data Lead)*\n\n## Objective: \n- Load and clean COVID-19 data\n- Transform data structure\n- Perform exploratory data analysis\n- Calculate key statistics\n'

In [42]:
# Cell 2: Imports and Setup
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Create Spark Session
spark = SparkSession.builder.appName("COVID19_Data_Prep").getOrCreate()

print("Spark session created successfully!")


Spark session created successfully!


In [43]:
# Cell 3: Data Loading
"""
## Step 1: Load Raw Data
"""
# Load the COVID-19 dataset
df_raw = spark.read.csv("C:/Users/HP/OneDrive/Desktop/2023-11-18.csv", header=True, inferSchema=True)

print(f"Dataset loaded: {df_raw.count()} rows")
print("\nSchema:")
df_raw.printSchema()
print("\nFirst 5 rows:")
df_raw.show(5)


Dataset loaded: 289 rows

Schema:
root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 02-01-2020: integer (nullable = true)
 |-- 02-02-2020: integer (nullable = true)
 |-- 02-03-2020: integer (nullable = true)
 |-- 02-04-2020: integer (nullable = true)
 |-- 02-05-2020: integer (nullable = true)
 |-- 02-06-2020: integer (nullable = true)
 |-- 02-07-2020: integer (nullable = true)
 |-- 02-08-2020: integer (nullable = true)
 |-- 02-09-2020: integer (nullable = true)
 |-- 02-10-

In [44]:
# Cell 4: Data Cleaning
"""
## Step 2: Data Cleaning
- Remove null values
- Filter invalid entries
- Standardize formats
"""
# Check for null values
print("Checking for null values in key columns:")
df_raw.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) 
               for c in ['Country/Region', 'Lat', 'Long']]).show()

# Clean data - remove rows with missing location info
df_clean = df_raw.filter(
    F.col("Country/Region").isNotNull() &
    F.col("Lat").isNotNull() &
    F.col("Long").isNotNull()
)

print(f"After cleaning: {df_clean.count()} rows")


Checking for null values in key columns:
+--------------+---+----+
|Country/Region|Lat|Long|
+--------------+---+----+
|             0|  2|   2|
+--------------+---+----+

After cleaning: 287 rows


In [45]:
# Cell 5: Data Transformation
"""
## Step 3: Transform Data Structure
Converting wide format (dates as columns) to long format (dates as rows)
"""
# Get all date columns (skip first 4 metadata columns)
date_cols = df_clean.columns[4:]
print(f"Number of date columns: {len(date_cols)}")

# Create unpivot expression
expressions = [f"'{col}', `{col}`" for col in date_cols]
stack_expr = f"stack({len(date_cols)}, {', '.join(expressions)}) as (DateStr, Cases)"

# Transform to long format
df_long = df_clean.select(
    F.col("`Country/Region`").alias("Country"),
    F.expr(stack_expr)
)

print("Data transformed to long format:")
df_long.show(10)
print("Columns:", df_long.columns)

Number of date columns: 1143
Data transformed to long format:
+-----------+-------+-----+
|    Country|DateStr|Cases|
+-----------+-------+-----+
|Afghanistan|1/22/20|    0|
|Afghanistan|1/23/20|    0|
|Afghanistan|1/24/20|    0|
|Afghanistan|1/25/20|    0|
|Afghanistan|1/26/20|    0|
|Afghanistan|1/27/20|    0|
|Afghanistan|1/28/20|    0|
|Afghanistan|1/29/20|    0|
|Afghanistan|1/30/20|    0|
|Afghanistan|1/31/20|    0|
+-----------+-------+-----+
only showing top 10 rows
Columns: ['Country', 'DateStr', 'Cases']


In [46]:
# Cell 6: Feature Engineering
"""
## Step 4: Calculate Daily New Cases
"""
from pyspark.sql.window import Window

# Window specification: partition by country, order by DATE STRING 
window_spec = Window.partitionBy("Country").orderBy("DateStr")

# Calculate daily new cases
df_processed = df_long.withColumn(
    "DailyNewCases",
    F.col("Cases") - F.lag("Cases", 1).over(window_spec)
)

# Handle nulls and negatives
df_processed = df_processed.fillna({"DailyNewCases": 0})
df_processed = df_processed.withColumn(
    "DailyNewCases",
    F.when(F.col("DailyNewCases") < 0, 0).otherwise(F.col("DailyNewCases"))
)

# Rename for clarity
df_processed = df_processed.withColumnRenamed("Cases", "TotalCases")

print("Daily cases calculated:")
df_processed.show(10)

Daily cases calculated:
+-----------+----------+----------+-------------+
|    Country|   DateStr|TotalCases|DailyNewCases|
+-----------+----------+----------+-------------+
|Afghanistan|01-01-2021|     52513|            0|
|Afghanistan|01-01-2022|    158107|       105594|
|Afghanistan|01-01-2023|    207616|        49509|
|Afghanistan|01-02-2021|     52586|            0|
|Afghanistan|01-02-2022|    158189|       105603|
|Afghanistan|01-02-2023|    207627|        49438|
|Afghanistan|01-03-2021|     52709|            0|
|Afghanistan|01-03-2022|    158183|       105474|
|Afghanistan|01-03-2023|    207654|        49471|
|Afghanistan|01-04-2021|     52909|            0|
+-----------+----------+----------+-------------+
only showing top 10 rows


In [47]:
# Cell 7: Exploratory Data Analysis
"""
## Step 5: EDA - Summary Statistics by Country
"""

# Calculate statistics per country
country_stats = df_processed.groupBy("Country") \
    .agg(
        F.sum("TotalCases").alias("TotalCasesSum"),
        F.max("TotalCases").alias("MaxTotalCases"),
        F.avg("DailyNewCases").alias("AvgDailyNewCases"),
        F.max("DailyNewCases").alias("PeakDailyNewCases")
    ).orderBy(F.desc("TotalCasesSum"))

print("Top 20 countries by total cases:")
country_stats.show(20)

Top 20 countries by total cases:
+--------------+-------------+-------------+--------------------+-----------------+
|       Country|TotalCasesSum|MaxTotalCases|    AvgDailyNewCases|PeakDailyNewCases|
+--------------+-------------+-------------+--------------------+-----------------+
|            US|  53813184406|    103802702| 2.770485602624672E7|         56533117|
|         India|  29131119694|     44690738|1.2931067160104986E7|         31851003|
|        Brazil|  21182690594|     37081209|   9852079.180227472|         18283244|
|        France|  16105911886|     38618509|    1153260.62335958|         38615082|
|       Germany|  13686043720|     38249060|   9888854.066491688|         31128276|
|United Kingdom|  12118271679|     24425309|   703788.4485272674|         24418748|
|        Russia|  10578569842|     22075858|   5968628.092738408|         13297910|
|         Italy|  10083161678|     25603510|   6577479.509186352|         19575854|
|        Turkey|   8840742699|     17042722

In [48]:
# Cell 8: Monthly Trends Analysis
"""
## Step 6: Analyze Monthly Trends
"""

print("\n=== Summary Statistics ===")
print(f"Countries analyzed: {country_stats.count()}")
print(f"Total global cases: {df_processed.agg(F.max('TotalCases')).collect()[0][0]:,.0f}")

# Show distribution of cases
print("\nCase Distribution:")
df_processed.select("DailyNewCases") \
           .summary("count", "mean", "stddev", "min", "max") \
           .show()

print("✅ Exploratory Data Analysis Complete!")


=== Summary Statistics ===
Countries analyzed: 201
Total global cases: 103,802,702

Case Distribution:
+-------+------------------+
|summary|     DailyNewCases|
+-------+------------------+
|  count|            328041|
|   mean| 654907.5108477294|
| stddev|3257117.3906031228|
|    min|                 0|
|    max|          56533117|
+-------+------------------+

✅ Exploratory Data Analysis Complete!


In [49]:
# Cell 9: Save to current folder
df_processed.toPandas().to_csv("cleaned_data.csv", index=False)
print("✅ File saved as 'cleaned_data.csv' in current folder")

✅ File saved as 'cleaned_data.csv' in current folder
